In [2]:
cd ..

E:\gdrive\Projects\Porto_Seguro\stacking


In [3]:
from utils import layer1
from utils.layer2 import get_meta_features
from utils.gini import *
from utils.utils import *

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [4]:
train = pd.read_csv('data/train.csv', na_values=-1)

In [7]:
train.ps_car_13.round(2)

0         0.88
1         0.62
2         0.64
3         0.54
4         0.57
5         0.88
6         0.64
7         0.90
8         0.78
9         1.46
10        0.88
11        0.95
12        0.77
13        0.98
14        0.57
15        1.72
16        0.76
17        0.88
18        0.67
19        0.46
20        0.57
21        0.58
22        0.98
23        0.73
24        0.72
25        0.89
26        1.31
27        0.68
28        0.72
29        0.64
          ... 
595182    0.73
595183    0.70
595184    0.66
595185    0.78
595186    0.85
595187    0.70
595188    0.80
595189    0.69
595190    0.51
595191    0.93
595192    0.79
595193    1.08
595194    0.71
595195    1.02
595196    0.69
595197    0.90
595198    0.84
595199    1.06
595200    0.79
595201    0.61
595202    0.74
595203    1.14
595204    0.80
595205    0.60
595206    0.90
595207    0.68
595208    0.97
595209    0.60
595210    0.76
595211    0.93
Name: ps_car_13, Length: 595212, dtype: float64

In [143]:
# train = train.fillna(train.median())
train = train.drop(train.columns[train.columns.str.startswith('ps_calc')], axis=1)
train = train.drop(['ps_ind_10_bin','ps_ind_11_bin','ps_ind_12_bin','ps_ind_13_bin',
                'ps_ind_18_bin','ps_car_03_cat', 'ps_car_10_cat'], axis=1)

In [144]:
train['ps_ind_0609'] = np.zeros_like(train.id)
train['ps_ind_0609'][train.ps_ind_06_bin==1] = 1
train['ps_ind_0609'][train.ps_ind_07_bin==1] = 2
train['ps_ind_0609'][train.ps_ind_08_bin==1] = 3
train['ps_ind_0609'][train.ps_ind_09_bin==1] = 4
train['ps_ind_0609'][train.ps_ind_0609==0] = 5

train.drop(train.loc[:,'ps_ind_06_bin':'ps_ind_09_bin'].columns, axis=1, inplace=True)

C:\Users\Zigkie\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Zigkie\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Zigkie\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Zigkie\Anaconda3\

In [145]:
train['ps_ind_1618'] = np.zeros_like(train.id)
train['ps_ind_1618'][train.ps_ind_16_bin==1] = 1
train['ps_ind_1618'][train.ps_ind_17_bin==1] = 2
train['ps_ind_1618'][train.ps_ind_1618==0] = 3

train.drop(['ps_ind_16_bin','ps_ind_17_bin'], axis=1, inplace=True)

C:\Users\Zigkie\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Zigkie\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Zigkie\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [146]:
train['ps_car_13_squared'] = (train.ps_car_13**2*48400).round(0)
train['ps_car_12_squared'] = (train.ps_car_12**2*10000).round(4)
train['ps_car_13_reg_03'] = train.ps_car_13*train.ps_reg_03

In [147]:
train['row_std'] = train.iloc[:, 2:].std(axis=1)
train['row_mean'] = train.iloc[:, 2:].mean(axis=1)
train['row_median'] = train.iloc[:, 2:].median(axis=1)
train['row_zero'] = (train == 0).sum(axis=1)
train['row_na'] = train.isnull().sum(axis=1)

In [148]:
median_cat = train[train.columns[~(train.columns.str.endswith('bin'))][2:]]
median_cat = (median_cat>median_cat.median(axis=0)).astype('int')
median_cat.columns = [str(col)+'_med' for col in median_cat.columns]

In [149]:
mean_cat = train[train.columns[~(train.columns.str.endswith('bin'))][2:]]
mean_cat = (mean_cat>mean_cat.mean(axis=0)).astype('int')
mean_cat.columns = [str(col)+'_mean' for col in mean_cat.columns]

In [151]:
train = pd.concat([train, median_cat, mean_cat], axis=1)

In [153]:
train.shape

(595212, 172)

In [154]:
train.columns.values

array(['id', 'target', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03',
       'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_ind_14', 'ps_ind_15',
       'ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'ps_car_01_cat',
       'ps_car_02_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat',
       'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_11_cat',
       'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15',
       'ps_ind_0609', 'ps_ind_1618', 'ps_car_13_squared',
       'ps_car_12_squared', 'ps_car_13_reg_03', 'row_std', 'row_mean',
       'row_median', 'row_zero', 'row_na', 'ps_ind_01_med',
       'ps_ind_02_cat_med', 'ps_ind_03_med', 'ps_ind_04_cat_med',
       'ps_ind_05_cat_med', 'ps_ind_14_med', 'ps_ind_15_med',
       'ps_reg_01_med', 'ps_reg_02_med', 'ps_reg_03_med',
       'ps_car_01_cat_med', 'ps_car_02_cat_med', 'ps_car_04_cat_med',
       'ps_car_05_cat_med', 'ps_car_06_cat_med', 'ps_car_07_cat_med',
       'ps_car_08_cat_med', 'ps_car_09_cat_med', 'ps_car_11_cat_med